In [37]:
import boto3, botocore
from botocore.exceptions import ClientError
import os, time, json, io, zipfile, base64
from datetime import date
from dotenv import load_dotenv


from misc import load_from_yaml, save_to_yaml
import iam, s3, lf, rds, vpc, ec2

load_dotenv(".env")
# boto3.setup_default_session(profile_name="AMominNJ")

True

In [2]:
ALL_IN_ONE_SG     = 'sg-0d8a868137f653df6'
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ['AWS_DEFAULT_REGION']
VPC_ID            = os.environ['AWS_DEFAULT_VPC']
SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
SUBNET_ID         = SUBNET_IDS[0]
AWS_INSTANCE_ID_JMASTER   = os.environ['AWS_INSTANCE_ID_JMASTER']
AWS_INSTANCE_ID_JAGENT    = os.environ['AWS_INSTANCE_ID_JAGENT']
AWS_DEFAULT_IMAGE_ID      = os.environ['AWS_DEFAULT_IMAGE_ID']
AWS_DEFAULT_KEY_PAIR_NAME = os.environ['AWS_DEFAULT_KEY_PAIR_NAME']
AWS_DEFAULT_INSTANCE_TYPE = os.environ['AWS_DEFAULT_INSTANCE_TYPE']

In [60]:
sts_client           = boto3.client('sts')
rds_client           = boto3.client('rds')
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
stepfunctions_client = boto3.client('stepfunctions')
apigateway_client    = boto3.client('apigateway')
lsn_client           = boto3.client('lambda')
events_client        = boto3.client('events')
sqs_client           = boto3.client('sqs')
emr_client = boto3.client('emr', region_name=REGION)

iam_client           = boto3.client('iam')
elbv2_client = boto3.client("elbv2", region_name=REGION)
autoscaling_client = boto3.client("autoscaling", region_name=REGION)
acm_client = boto3.client("acm", region_name=REGION)  # Change region as needed
route53 = boto3.client("route53", region_name=REGION)

In [ ]:
ec2_client           = boto3.client('ec2', region_name=REGION)
ec2_resource         = boto3.resource('ec2', region_name=REGION)

# # Example: Get a specific VPC
# vpc = ec2_resource.Vpc('vpc_id')

# # Example: Get a specific EBS volume
# volume = ec2_resource.Volume('volume_id')

[boto3 doc: ELB](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/elb.html)
[boto3 doc: ELBv2](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/elbv2.html)

##   [Auto Scaling and Load Balancing on AWS](https://www.youtube.com/watch?v=0mwgbiJae5Q&list=PLO95rE9ahzRs0QMA8qtIAstWFo4X4gHtH&index=5)

write a python script to create a AWS Auto Scalling Group with all possible option. please write the script without defining any python function. create all resources step by step using boto3 API. 

In [ ]:
vpc_cidr_block = '172.0.0.0/16'
vpc_name = 'httx-asg-alb-vpc'
vpc_id = ec2_client.create_vpc(CidrBlock=vpc_cidr_block)['Vpc']['VpcId']
# vpc = ec2_resource.Vpc('vpc_id')


In [32]:
# Create Internet Gateway and attach that with VPC
igw = ec2_resource.create_internet_gateway()

In [ ]:
print(igw.id)

igw-0e7b77977cdf6f731


In [ ]:
ec2_client.attach_internet_gateway(InternetGatewayId=igw.id,VpcId=vpc_id)

##### Subnets

In [23]:
subnet_configs = [
    {'cidr_block': '172.0.1.0/24', 'az': 'us-east-1a', 'tag': 'public-subnet-01'},
    {'cidr_block': '172.0.2.0/24', 'az': 'us-east-1b', 'tag': 'public-subnet-02'},
    {'cidr_block': '172.0.3.0/24', 'az': 'us-east-1a', 'tag': 'private-subnet-01'},
    {'cidr_block': '172.0.4.0/24', 'az': 'us-east-1b', 'tag': 'private-subnet-02'}
]

In [ ]:
public_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[0]['cidr_block'],
    VpcId=vpc_id,
    AvailabilityZone=subnet_configs[0]['az']
)
ec2_client.create_tags(Resources=[public_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[0]['tag']}])

public_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[1]['cidr_block'],
    VpcId=vpc_id,
    AvailabilityZone=subnet_configs[1]['az']
)
ec2_client.create_tags(Resources=[public_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[1]['tag']}])

 -  **Need to `enable auto-assign public IPv4 address`**:
    -   `public_subnet1` --> `Actions` --> `Edit Subnet Settings` --> `enable auto-assign public IPv4 address` --> `Save`
    -   `public_subnet2` --> `Actions` --> `Edit Subnet Settings` --> `enable auto-assign public IPv4 address` --> `Save`

In [ ]:
private_subnet1 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[2]['cidr_block'],
    VpcId=vpc_id,
    AvailabilityZone=subnet_configs[2]['az']
)
ec2_client.create_tags(Resources=[private_subnet1.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[2]['tag']}])

private_subnet2 = ec2_resource.create_subnet(
    CidrBlock=subnet_configs[3]['cidr_block'],
    VpcId=vpc_id,
    AvailabilityZone=subnet_configs[3]['az']
)
ec2_client.create_tags(Resources=[private_subnet2.id],Tags=[{'Key': 'Name', 'Value': subnet_configs[3]['tag']}])

##### Security Group

In [16]:
ALB1_RULE = [
    {
        "IpProtocol": "tcp",
        "FromPort": 80,
        "ToPort": 80,
        "IpRanges": [{"CidrIp": "0.0.0.0/0", "Description": "HTTP_Port"}],
    },
    {
        "IpProtocol": "tcp",
        "FromPort": 443,
        "ToPort": 443,
        "IpRanges": [{"CidrIp": "0.0.0.0/0", "Description": "HTTPS_Port"}],
    },
]

ALL_IN_ONE_OUTBOUND_RULES = [
    {
        'IpProtocol': '-1', # '-1' means all protocols
        'FromPort': -1,
        'ToPort': -1,
        'IpRanges': [{'CidrIp': '0.0.0.0/0'}]
    }
]

tags = [
    {
        'Key': 'Name',
        'Value': 'ALB1_SG'
    }
]


In [17]:
ALB1_SG_ID = ec2.create_security_group(
    "ALB1_SG_NAME",
    vpc_id,
    inbound_rules=ALB1_RULE,
    outbound_rules="",
    tags=tags,
    description="ALB1",
)["GroupId"]

Security Group 'sg-0786b90b3bd212897' is Created in VPC vpc-0787f9c66827a9980


In [18]:
WEB1_SG_RULE = [
    {
        "IpProtocol": "tcp",
        "FromPort": 80,
        "ToPort": 80,
        "UserIdGroupPairs": [
            {
                "GroupId": ALB1_SG_ID,
                "Description": "Allow all tcp traffic from another security group on port 80",
            }
        ],
    },
    {
        "IpProtocol": "tcp",
        "FromPort": 22, # SSH: For testing only
        "ToPort": 22,
        "IpRanges": [{"CidrIp": "0.0.0.0/0", "Description": "SSH_Port"}],
    },
    {
        "IpProtocol": "tcp",
        "FromPort": 80, # HTTP: For testing only
        "ToPort": 80,
        "IpRanges": [{"CidrIp": "0.0.0.0/0", "Description": "HTTP_Port"}],
    },
]

WEB1_SG_ID = ec2.create_security_group(
    "WEB1_SG_NAME",
    vpc_id,
    inbound_rules=WEB1_SG_RULE,
    outbound_rules="",
    tags=[{"Key": "Name", "Value": "WEB1_SG"}],
    description="WEB1_SG",
)["GroupId"]

Security Group 'sg-06a31d2e790e3e6c5' is Created in VPC vpc-0787f9c66827a9980


Error: An error occurred (InvalidParameterValue) when calling the AuthorizeSecurityGroupIngress operation: Invalid value for portRange. Must specify both from and to ports with TCP/UDP.


##### Target Groups

In [22]:
# Define the target group parameters
TARGET_GROUP_NAME = "TG1"

target_group = elbv2_client.create_target_group(
    Name=TARGET_GROUP_NAME,
    Protocol="HTTP",  # Options: HTTP, HTTPS, TCP, TLS, UDP, TCP_UDP, GENEVE
    Port=80,  # Port on which the target is listening
    VpcId=vpc_id,
    HealthCheckProtocol="HTTP",  # Options: HTTP, HTTPS, TCP, TLS
    HealthCheckPort="traffic-port",  # Use "traffic-port" to match the listener port
    HealthCheckEnabled=True,  # Enable health checks
    HealthCheckPath="/",  # Path for health check requests
    HealthCheckIntervalSeconds=30,  # Interval in seconds between health checks
    HealthCheckTimeoutSeconds=5,  # Timeout before considering the check as failed
    HealthyThresholdCount=3,  # Number of successful checks to consider healthy
    UnhealthyThresholdCount=2,  # Number of failed checks to consider unhealthy
    Matcher={"HttpCode": "200-299"},  # HTTP response code for a successful health check
    TargetType="instance",  # Options: instance, ip, lambda, alb
    Tags=[{"Key": "Environment", "Value": "Production"}],  # Optional tags
)

# Extract the Target Group ARN
TARGET_GROUP_ARN = target_group["TargetGroups"][0]["TargetGroupArn"]
print(f"Target Group created: {TARGET_GROUP_ARN}")

## We will be attaching the Target Group to the Auto Scaling Group in the next step.


Target Group created: arn:aws:elasticloadbalancing:us-east-1:381492255899:targetgroup/TG1/6848ff432ca636ce


##### ACM

In [ ]:
# Initialize ACM client
client = boto3.client("acm", region_name="us-east-1")  # Change region as needed

# Define certificate details
domain_name = "harnesstechtx.com"  # Replace with your actual domain
subject_alternative_names = ["www.harnesstechtx.com", "site.harnesstechtx.com"]  # SANs
validation_method = "DNS"  # Options: "DNS" or "EMAIL"
tags = [{"Key": "Environment", "Value": "QA"}]  # Optional tags
options = {"CertificateTransparencyLoggingPreference": "ENABLED"}  # Enable logging

# Request the ACM Certificate
ssl_certificate_response = client.request_certificate(
    DomainName=domain_name,
    ValidationMethod=validation_method,
    SubjectAlternativeNames=subject_alternative_names,
    IdempotencyToken="unique-token-123",  # Replace with a unique identifier
    Options=options,
    Tags=tags,
)

# Extract Certificate ARN
SSL_CERTIFICATE_ARN = ssl_certificate_response["CertificateArn"]


##### Load Balancer

In [ ]:
# ec2_client.modify_vpc_attribute(VpcId=vpc_id, EnableDnsSupport={"Value": True})
# ec2_client.modify_vpc_attribute(VpcId=vpc_id, EnableDnsHostnames={"Value": True})

# ALB_SSL_ROLE_NAME = "ALB-SSL-Role"

# # Step 4: Create an IAM Role and Policy for SSL (Optional, needed for HTTPS termination)
# ALB_SSL_ROLE_ARN = iam_client.create_role(
#     RoleName=ALB_SSL_ROLE_NAME,
#     AssumeRolePolicyDocument="""{
#         "Version": "2012-10-17",
#         "Statement": [
#             {
#                 "Effect": "Allow",
#                 "Principal": {
#                     "Service": "elasticloadbalancing.amazonaws.com"
#                 },
#                 "Action": "sts:AssumeRole"
#             }
#         ]
#     }""",
# )["Role"]["Arn"]

# iam_client.attach_role_policy(
#     RoleName=ALB_SSL_ROLE_NAME,
#     PolicyArn="arn:aws:iam::aws:policy/service-role/AmazonEC2ContainerServiceRole",
# )

In [ ]:
# Step 7: Create Application Load Balancer
alb_response = elbv2_client.create_load_balancer(
    Name="ALB1",
    Subnets=[public_subnet1.id, public_subnet2.id],
    SecurityGroups=[ALB1_SG_ID],
    Scheme="internet-facing",  # Options: internet-facing, internal
    Type="application",
    IpAddressType="ipv4",  # Options: ipv4, dualstack
)
ALB_ARN = alb_response["LoadBalancers"][0]["LoadBalancerArn"]
ALB_DNS = alb_response["LoadBalancers"][0]["DNSName"]


# Step 8: Create HTTP Listener (Specify the Target Group)
http_listener_response = elbv2_client.create_listener(
    LoadBalancerArn=ALB_ARN,
    Protocol="HTTP",
    Port=80,
    DefaultActions=[{"Type": "forward", "TargetGroupArn": TARGET_GROUP_ARN}],
)
HTTP_LISTENER_ARN = http_listener_response["Listeners"][0]["ListenerArn"]

In [36]:
print(ALB_ARN, ALB_DNS, HTTP_LISTENER_ARN, sep="\n")

arn:aws:elasticloadbalancing:us-east-1:381492255899:loadbalancer/app/ALB1/ccd548fb7b0feb10
ALB1-1782892738.us-east-1.elb.amazonaws.com
arn:aws:elasticloadbalancing:us-east-1:381492255899:listener/app/ALB1/ccd548fb7b0feb10/2d75b1ed6da2e498


In [ ]:

# # Step 9: Create HTTPS Listener (Requires an SSL Certificate)
# https_listener_response = elbv2_client.create_listener(
#     LoadBalancerArn=ALB_ARN,
#     Protocol="HTTPS",
#     Port=443,
#     SslPolicy="ELBSecurityPolicy-2016-08",
#     Certificates=[{"CertificateArn": SSL_CERTIFICATE_ARN}],
#     DefaultActions=[{"Type": "forward", "TargetGroupArn": TARGET_GROUP_ARN}],
# )
# HTTPS_LISTENER_ARN = https_listener_response["Listeners"][0]["ListenerArn"]


# # Step 10: Wait until ALB is active
# while True:
#     alb_status = elbv2_client.describe_load_balancers(LoadBalancerArns=[ALB_ARN])
#     state = alb_status["LoadBalancers"][0]["State"]["Code"]
#     if state == "active":
#         break
#     print(f"Waiting for ALB to become active... Current state: {state}")
#     time.sleep(10)

# print("ALB is now active and ready to use!")

# # Step 11: Output Load Balancer DNS
# print(f"ALB DNS Name: {ALB_DNS}")


##### Launch Template

In [ ]:
# Define Launch Template parameters
launch_template_name = "MyLT"
ami_id = "ami-053a45fff0a704a47"  # Amazon Linux 2023 AMI 2023.6.20250211.0 x86_64 HVM kernel-6.1
instance_type = "t2.micro"

# User Data (Base64 Encoded)
user_data_script = """#!/bin/bash

# Update the system and install Apache
yum update -y
yum install -y httpd

# Start and enable Apache to run on boot
systemctl start httpd
systemctl enable httpd

# Fetch the latest token for accessing EC2 instance metadata
TOKEN=$(curl -X PUT "http://169.254.169.254/latest/api/token" -H "X-aws-ec2-metadata-token-ttl-seconds: 21600")

# Fetch the instance ID
INSTANCEID=$(curl -s http://169.254.169.254/latest/meta-data/instance-id -H "X-aws-ec2-metadata-token: $TOKEN")

# Create or overwrite `index.html` with the instance ID information
echo "<center><h1>This instance has the ID: $INSTANCEID </h1></center>" > /var/www/html/index.html

# Ensure httpd can read the `index.html` file and its directory
chown apache:apache /var/www/html/index.html
chmod 755 /var/www/html
chmod 644 /var/www/html/index.html

# Restart Apache to apply changes
systemctl restart httpd
"""
user_data_encoded = base64.b64encode(user_data_script.encode("utf-8")).decode("utf-8")

# Define Block Device Mappings
block_device_mappings = [
    {
        "DeviceName": "/dev/xvda",
        "Ebs": {
            "VolumeSize": 20,  # 20 GB Root Volume
            "VolumeType": "gp3",
            "DeleteOnTermination": True,
            "Encrypted": True,
        },
    },
    {
        "DeviceName": "/dev/xvdb",
        "Ebs": {
            "VolumeSize": 50,  # Additional 50 GB Volume
            "VolumeType": "gp3",
            "DeleteOnTermination": False,
            "Encrypted": False,
        },
    },
]

# Create Launch Template
response = ec2_client.create_launch_template(
    LaunchTemplateName=launch_template_name,
    VersionDescription="Initial Version",
    LaunchTemplateData={
        "ImageId": ami_id,
        "InstanceType": instance_type,
        "SecurityGroupIds": [WEB1_SG_ID],
        "UserData": user_data_encoded,
        "BlockDeviceMappings": block_device_mappings,
        # "NetworkInterfaces": [
        #     {
        #         "SubnetId": SUBNET_ID,
        #         "DeviceIndex": 0,
        #         "AssociatePublicIpAddress": True,
        #         "Groups": [WEB1_SG_ID],
        #     }
        # ],
        "TagSpecifications": [
            {
                "ResourceType": "instance",
                "Tags": [{"Key": "Environment", "Value": "QA"}],
            }
        ],
        "Monitoring": {"Enabled": True},  # Enable detailed monitoring
        "DisableApiTermination": False,  # Allow instance termination
        "InstanceInitiatedShutdownBehavior": "stop",  # 'terminate' or 'stop'
    },
)

# Extract Launch Template ID & ARN
LAUNCH_TEMPLATE_ID = response["LaunchTemplate"]["LaunchTemplateId"]
print(LAUNCH_TEMPLATE_ID)

lt-008808c40faac42fa


##### Autoscaling Group

In [ ]:
ASG_NAME = "MyASG"

# Step 2: Create Auto Scaling Group
asg_response = autoscaling_client.create_auto_scaling_group(
    AutoScalingGroupName=ASG_NAME,
    LaunchTemplate={"LaunchTemplateId": LAUNCH_TEMPLATE_ID, "Version": "$Latest"},
    MinSize=2, # Minimum number of instances
    MaxSize=4, # Maximum number of instances
    DesiredCapacity=2,
    VPCZoneIdentifier=",".join([public_subnet1.id, public_subnet2.id]), #
    HealthCheckType="EC2", # Options: EC2, ELB
    HealthCheckGracePeriod=300, # Wait time before checking health status
    Tags=[{"Key": "Environment", "Value": "Production", "PropagateAtLaunch": True}],
    NewInstancesProtectedFromScaleIn=True, # Protect new instances from scale in
    DefaultCooldown=300, # Default cooldown period in seconds
    AvailabilityZones=["us-east-1a", "us-east-1b"],
    TerminationPolicies=["Default"], # Options: Default, OldestInstance, NewestInstance, OldestLaunchConfiguration, ClosestToNextInstanceHour, AllocationStrategy
    TargetGroupARNs=[TARGET_GROUP_ARN],  # Add ALB target group ARNs if applicable
    InstanceMaintenancePolicy={"MinHealthyPercentage": 50, "MaxHealthyPercentage": 100}, # Instance maintenance policy
)

In [52]:
# Extract required parts
alb_name, alb_id = ALB_ARN.split("/")[-2:]
tg_name, tg_id = TARGET_GROUP_ARN.split("/")[-2:]

# Construct the correct Resource Label
resource_label = f"app/{alb_name}/{alb_id}/targetgroup/{tg_name}/{tg_id}"


# Step 3: Attach Scaling Policies
scaling_policy_response = autoscaling_client.put_scaling_policy(
    AutoScalingGroupName=ASG_NAME,
    PolicyName="TargetTrackingPolicy",
    PolicyType="TargetTrackingScaling",
    TargetTrackingConfiguration={
        "PredefinedMetricSpecification": {
            "PredefinedMetricType": "ALBRequestCountPerTarget",  # ASGAverageCPUUtilization, ASGAverageNetworkIn, ASGAverageNetworkOut, ALBRequestCountPerTarget
            "ResourceLabel": resource_label,  # Correct format

        },
        "TargetValue": 50.0,
    },
)
print("Scaling Policy Created:", scaling_policy_response["PolicyARN"])

# # Step 4: Attach Notifications (Optional)
# autoscaling_client.put_notification_configuration(
#     AutoScalingGroupName=ASG_NAME,
#     TopicARN="arn:aws:sns:us-east-1:123456789012:MySNSTopic",
#     NotificationTypes=[
#         "autoscaling:EC2_INSTANCE_LAUNCH",
#         "autoscaling:EC2_INSTANCE_TERMINATE",
#     ],
# )
# print("Notification Configurations Set")

# Step 5: Verify Auto Scaling Group Status
asg_status = autoscaling_client.describe_auto_scaling_groups(
    AutoScalingGroupNames=[ASG_NAME]
)
print("Auto Scaling Group Details:", asg_status)


Scaling Policy Created: arn:aws:autoscaling:us-east-1:381492255899:scalingPolicy:8800f825-d056-4f50-86c2-a424b5085569:autoScalingGroupName/MyASG:policyName/TargetTrackingPolicy
Auto Scaling Group Details: {'AutoScalingGroups': [{'AutoScalingGroupName': 'MyASG', 'AutoScalingGroupARN': 'arn:aws:autoscaling:us-east-1:381492255899:autoScalingGroup:dea29129-8d8b-4d4d-93e4-447f5c52c51e:autoScalingGroupName/MyASG', 'LaunchTemplate': {'LaunchTemplateId': 'lt-008808c40faac42fa', 'LaunchTemplateName': 'MyLT', 'Version': '$Latest'}, 'MinSize': 1, 'MaxSize': 4, 'DesiredCapacity': 2, 'DefaultCooldown': 300, 'AvailabilityZones': ['us-east-1a', 'us-east-1b'], 'LoadBalancerNames': [], 'TargetGroupARNs': ['arn:aws:elasticloadbalancing:us-east-1:381492255899:targetgroup/TG1/6848ff432ca636ce'], 'HealthCheckType': 'EC2', 'HealthCheckGracePeriod': 300, 'Instances': [{'InstanceId': 'i-04d9cc500452b14e9', 'InstanceType': 't2.micro', 'AvailabilityZone': 'us-east-1b', 'LifecycleState': 'InService', 'HealthStat

##### Delete Resources

In [ ]:
ec2.delete_security_group(WEB1_SG_ID)
ec2.delete_security_group(ALB1_SG_ID)

Deleted Security Group: sg-072e783262d750070


In [57]:
elbv2_client.delete_target_group(TargetGroupArn=TARGET_GROUP_ARN)

{'ResponseMetadata': {'RequestId': 'fb0be7fb-45ab-461d-be19-576f88a69259',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fb0be7fb-45ab-461d-be19-576f88a69259',
   'content-type': 'text/xml',
   'content-length': '259',
   'date': 'Fri, 14 Feb 2025 06:00:21 GMT'},
  'RetryAttempts': 0}}

In [56]:
elbv2_client.delete_load_balancer(LoadBalancerArn=ALB_ARN)

{'ResponseMetadata': {'RequestId': '5225a004-9da6-4c68-91b3-def710b2abab',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '5225a004-9da6-4c68-91b3-def710b2abab',
   'content-type': 'text/xml',
   'content-length': '262',
   'date': 'Fri, 14 Feb 2025 05:59:54 GMT'},
  'RetryAttempts': 0}}

In [54]:
ec2_client.delete_launch_template(LaunchTemplateId=LAUNCH_TEMPLATE_ID)

{'LaunchTemplate': {'LaunchTemplateId': 'lt-008808c40faac42fa',
  'LaunchTemplateName': 'MyLT',
  'CreateTime': datetime.datetime(2025, 2, 14, 5, 6, 14, tzinfo=tzutc()),
  'CreatedBy': 'arn:aws:iam::381492255899:user/AMominNJ',
  'DefaultVersionNumber': 1,
  'LatestVersionNumber': 1,
  'Operator': {'Managed': False}},
 'ResponseMetadata': {'RequestId': 'f2cc9380-170d-4cec-8738-d211c476be7f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'f2cc9380-170d-4cec-8738-d211c476be7f',
   'cache-control': 'no-cache, no-store',
   'strict-transport-security': 'max-age=31536000; includeSubDomains',
   'vary': 'accept-encoding',
   'content-type': 'text/xml;charset=UTF-8',
   'transfer-encoding': 'chunked',
   'date': 'Fri, 14 Feb 2025 05:59:22 GMT',
   'server': 'AmazonEC2'},
  'RetryAttempts': 0}}

In [53]:
autoscaling_client.delete_auto_scaling_group(AutoScalingGroupName=ASG_NAME,ForceDelete=True)

{'ResponseMetadata': {'RequestId': 'e87ff5fc-1dcd-45d4-a626-02b76ef94baf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e87ff5fc-1dcd-45d4-a626-02b76ef94baf',
   'content-type': 'text/xml',
   'content-length': '231',
   'date': 'Fri, 14 Feb 2025 05:59:17 GMT'},
  'RetryAttempts': 0}}

In [58]:
vpc.delete_vpc_with_dependencies(vpc_id)

An error occurred: An error occurred (DependencyViolation) when calling the DetachInternetGateway operation: Network vpc-0787f9c66827a9980 has some mapped public address(es). Please unmap those public address(es) before detaching the gateway.


False

## [ Securing AWS Load Balancers with SSL/TLS - Hands-on](https://www.youtube.com/watch?v=R-x1qudp1ig&list=PLO95rE9ahzRs0QMA8qtIAstWFo4X4gHtH&index=6&t=619s)

In [59]:
import boto3
import time
import json

# Initialize the CloudFormation client
cloudformation = boto3.client("cloudformation", region_name="us-east-1")  # Change as needed

# Read CloudFormation template from a file
with open("/Users/am/mydocs/Software_Development/Web_Development/aws/auto_scaling_and_load_balancing/alb-cf-template.yml", "r") as file:
    template_body = file.read()

# Define stack name
stack_name = "secure-listner-lab"

# Define CloudFormation parameters
parameters = [
    {"ParameterKey": "VPCId", "ParameterValue": "vpc-03617a8a518caa526"},  # Replace with actual VPC ID
    {"ParameterKey": "SubnetIdOne", "ParameterValue": "subnet-0a972b05a5b162feb"},  # default-subnet-a
    {"ParameterKey": "SubnetIdTwo", "ParameterValue": "subnet-0ca765b361e4cb186"},  # default-subnet-c
    {"ParameterKey": "ImageId", "ParameterValue": "ami-053a45fff0a704a47"},  # Replace with valid AMI ID
    {"ParameterKey": "InstanceType", "ParameterValue": "t2.micro"}
]

# Create CloudFormation stack
print(f"Creating CloudFormation stack: {stack_name}")
response = cloudformation.create_stack(
    StackName=stack_name,
    TemplateBody=template_body,
    Parameters=parameters,
    Capabilities=["CAPABILITY_IAM", "CAPABILITY_NAMED_IAM"],  # Required for IAM resources
)


Creating CloudFormation stack: secure-listner-lab


In [62]:
hosted_zone_id = "Z04555692B7PI94BFJEBI"  # Replace with actual hosted zone ID

# Define certificate details
domain_name = "harnesstechtx.com"  # Replace with your domain
alternative_names = ["www.harnesstechtx.com", "sub.harnesstechtx.com"]  # Replace with alternative domain names


In [ ]:
alb_dns_name = "secure-LoadB-qNBBuPNmMEVs-303894070.us-east-1.elb.amazonaws.com."
elb_hosted_zone_id = "Z35SXDOTRQ7X7K"
# Define records
record_sets = [
    {
        "Action": "UPSERT",
        "ResourceRecordSet": {
            "Name": domain_name,
            "Type": "A",
            "TTL": 300,  # 300 seconds (5 minutes)
            "AliasTarget": {
                "HostedZoneId": elb_hosted_zone_id,  # Correct ELB Hosted Zone ID
                "DNSName": alb_dns_name,
                "EvaluateTargetHealth": True
            }
        }
    },
]

# Create Route 53 record set
print("Creating Route 53 records...")
response = route53.change_resource_record_sets(
    HostedZoneId=hosted_zone_id,  # Route 53 hosted zone
    ChangeBatch={
        "Comment": "Adding an alias record to Route 53",
        "Changes": record_sets
    }
)

print("Route 53 records created successfully.")
print("Change Info:", response["ChangeInfo"])

Creating Route 53 records...
Route 53 records created successfully.
Change Info: {'Id': '/change/C0118376HEYEAKW83EKQ', 'Status': 'PENDING', 'SubmittedAt': datetime.datetime(2025, 2, 15, 6, 8, 26, 746000, tzinfo=tzutc()), 'Comment': 'Adding an alias record to Route 53'}


#### Delete Resources

In [ ]:
acm_client.delete_certificate(CertificateArn=certificate_arn)